In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kstest, anderson, probplot
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler
import holidays

In [3]:
def check_normality(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        ks_stat, ks_p_value = kstest(df[col], 'norm', args=(df[col].mean(), df[col].std()))
        print(f'Kolmogorov-Smirnov test p-value: {ks_p_value}')
        if ks_p_value < 0.05:
            print(f'{col} is likely not normally distributed based on KS Test.\n')
        else:
            print(f'{col} is likely normally distributed based on KS Test.\n')
            
        # Anderson-Darling Test
        ad_result = anderson(df[col], dist='norm')
        print(f'Anderson-Darling Test Statistic: {ad_result.statistic}\n')
        for i in range(len(ad_result.critical_values)):
            print(f'Critical value for {ad_result.significance_level[i]}%: {ad_result.critical_values[i]}')
            if ad_result.statistic > ad_result.critical_values[i]:
                print(f'{col} is likely not normally distributed at {ad_result.significance_level[i]}% level.')
            else:
                print(f'{col} is likely normally distributed at {ad_result.significance_level[i]}% level.')
                
        # visual check for normality
        plt.figure(figsize=(8, 6))
        probplot(df[col], dist="norm", plot=plt)
        plt.title(f'Q-Q plot for {col}')
        plt.show()
            
        plt.figure(figsize=(8, 6))
        sns.histplot(df[col], kde=True)
        plt.title(f'Histogram for {col}')
        plt.show()
    

def check_outliers_iqr(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Identifying outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        print(f'Outliers in {col}:')
        print(outliers)
        

In [4]:
fpath_01 = '../datasets/raw/consumer_airfare/table1_top1000_contiguous_state_city-pair_markets.csv'
fpath_02 = '../datasets/raw/consumer_airfare/table1a_all_US_airport_pair_markets.csv'
fpath_03 = '../datasets/raw/consumer_airfare/table2_top1000_city-pair.csv'
fpath_04 = '../datasets/raw/consumer_airfare/table3_city-pair_markets_with_substantial_increase_inAvg.csv'
fpath_05 = '../datasets/raw/consumer_airfare/table4_city-pair_markets_with_substantial_decrease_inAvg.csv'
fpath_06 = '../datasets/raw/consumer_airfare/table5_detailed_fare_info_highest_lowest_fare_markets_under750Miles.csv'
fpath_07 = '../datasets/raw/consumer_airfare/table7_fare_premiums_for_select_cities_with_more_than_20_passengers_per_day.csv'
df1 = pd.read_csv(fpath_01, sep=';')
df1a = pd.read_csv(fpath_02, sep=';')
df2 = pd.read_csv(fpath_03, sep=';')
df3 = pd.read_csv(fpath_04, sep=';')
df4 = pd.read_csv(fpath_05, sep=';')
df5 = pd.read_csv(fpath_06, sep=';')
df7 = pd.read_csv(fpath_07, sep=';')
print('successfully loaded the data!')

successfully loaded the data!


In [5]:
dfs = [df1, df1a, df2, df3, df4, df5, df7]
for df in dfs:
    cols_to_drop = list(df.filter(regex='^tbl|^Geocoded_City'))
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
    df.shape
    df.info()
    print('--------------------------------------------------------')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115035 entries, 0 to 115034
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Year            115035 non-null  int64  
 1   quarter         115035 non-null  int64  
 2   citymarketid_1  115035 non-null  object 
 3   citymarketid_2  115035 non-null  int64  
 4   city1           115035 non-null  object 
 5   city2           115035 non-null  object 
 6   nsmiles         115035 non-null  int64  
 7   passengers      115035 non-null  object 
 8   fare            115035 non-null  float64
 9   carrier_lg      115035 non-null  object 
 10  large_ms        115035 non-null  float64
 11  fare_lg         115035 non-null  float64
 12  carrier_low     115032 non-null  object 
 13  lf_ms           115032 non-null  float64
 14  fare_low        115032 non-null  float64
 15  table_1_flag    115035 non-null  int64  
dtypes: float64(5), int64(5), object(6)
memory usage: 14.0+ M

In [6]:
for df in dfs:
    print(df.isna().sum())
    df.drop_duplicates(inplace=True)
    print('-------------------------------------------------------------------------------------------')

Year              0
quarter           0
citymarketid_1    0
citymarketid_2    0
city1             0
city2             0
nsmiles           0
passengers        0
fare              0
carrier_lg        0
large_ms          0
fare_lg           0
carrier_low       3
lf_ms             3
fare_low          3
table_1_flag      0
dtype: int64
-------------------------------------------------------------------------------------------
Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
airportid_1          0
airportid_2          0
airport_1            0
airport_2            0
nsmiles              0
passengers           0
fare                 0
carrier_lg        1550
large_ms          1550
fare_lg           1550
carrier_low       1622
lf_ms             1622
fare_low          1622
dtype: int64
-------------------------------------------------------------------------------------------
Year              0
quarter      

## drop irrelevant columns

In [7]:
df1.drop(columns=['table_1_flag'], inplace=True)

df1a.drop(columns=['airportid_1', 'airportid_2', 'airport_1', 'airport_2'], inplace=True)

df7.drop(columns=['cityname', 'airportid', 'apt', 'TotalMkts', 'SHMkts', 'SHPerLFMkts', 'LHMkts', 'LHPerLFMkts'], inplace=True)

## handling data types

In [8]:
df1['passengers'] = df1['passengers'].astype(str).str.replace(',', '', regex=False)
df1['passengers'] = pd.to_numeric(df1['passengers'], errors='coerce')
df1['passengers'] = df1['passengers'].astype(int)
df1.dtypes

Year                int64
quarter             int64
citymarketid_1     object
citymarketid_2      int64
city1              object
city2              object
nsmiles             int64
passengers          int64
fare              float64
carrier_lg         object
large_ms          float64
fare_lg           float64
carrier_low        object
lf_ms             float64
fare_low          float64
dtype: object

In [9]:
df1a_num_cols = ['fare', 'lf_ms', 'large_ms', 'fare_lg', 'fare_low']

df1a['passengers'] = df1a['passengers'].astype(str).str.replace(',', '', regex=False)
df1a['passengers'] = pd.to_numeric(df1a['passengers'], errors='coerce')
df1a['passengers'] = df1a['passengers'].astype(int)

df1a['nsmiles'] = df1a['nsmiles'].str.replace(',', '.', regex=False)  
df1a['nsmiles'] = df1a['nsmiles'].replace(['NA', 'N/A', 'None'], np.nan)  
df1a['nsmiles'] = pd.to_numeric(df1a['nsmiles'], errors='coerce')

for col in df1a_num_cols:
    if df1a[col].dtype == 'object':
        df1a[col] = df1a[col].astype(str).str.replace(',', '', regex=False)
        
        if col in ['large_ms', 'fare_lg', 'fare_low']:
            df1a[col] = df1a[col].replace(['NA', 'N/A', 'None'], np.nan)
        df1a[col] = pd.to_numeric(df1a[col], errors='coerce')
        
df1a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249884 entries, 0 to 249883
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Year            249884 non-null  int64  
 1   quarter         249884 non-null  int64  
 2   citymarketid_1  249884 non-null  int64  
 3   citymarketid_2  249884 non-null  int64  
 4   city1           249884 non-null  object 
 5   city2           249884 non-null  object 
 6   nsmiles         249884 non-null  float64
 7   passengers      249884 non-null  int64  
 8   fare            249884 non-null  float64
 9   carrier_lg      248334 non-null  object 
 10  large_ms        248334 non-null  float64
 11  fare_lg         248334 non-null  float64
 12  carrier_low     248262 non-null  object 
 13  lf_ms           248262 non-null  float64
 14  fare_low        248262 non-null  float64
dtypes: float64(6), int64(5), object(4)
memory usage: 28.6+ MB


In [10]:
for col in df2.columns:
    if any(x in col for x in ['passengers', 'distance']):
        if any(y in col for y in ['passengers']):
            df2[col] = df2[col].astype(str).str.replace(',', '', regex=True).str.strip()
            df2[col] = pd.to_numeric(df2[col], errors='coerce').astype(pd.Int64Dtype())
        else:
            df2[col] = df2[col].astype(str).str.replace(',', '', regex=False)
            df2[col] = df2[col].replace(['NA', 'N/A', 'None', np.inf, -np.inf], np.nan)
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
df2.dtypes

Year                int64
quarter             int64
citymarketid        int64
city               object
markets             int64
cur_passengers      Int64
cur_fare          float64
cur_yield         float64
distance          float64
ly_passengers       Int64
ly_fare           float64
ly_yield          float64
ly_distance       float64
dtype: object

In [11]:
for col in df3.columns:
    if col in ['amount_change_pax', 'cur_passengers', 'ly_passengers']:
        df3[col] = df3[col].astype(str).str.replace(',', '', regex=True).str.strip()
        df3[col] = pd.to_numeric(df3[col], errors='coerce')
        df3[col] = df3[col].astype(pd.Int64Dtype())

df3.dtypes

Year                   object
quarter                 int64
citymarketid_1         object
citymarketid_2         object
city1                  object
city2                  object
cur_passengers          Int64
cur_fare              float64
ly_fare               float64
ly_passengers           Int64
amount_change         float64
percent_change        float64
amount_change_pax       Int64
percent_change_pax    float64
dtype: object

In [12]:
for col in df4.columns:
    if col in ['amount_change_pax', 'cur_passengers', 'ly_passengers']:
        df4[col] = df4[col].astype(str).str.replace(',', '', regex=True).str.strip()
        df4[col] = pd.to_numeric(df4[col], errors='coerce')
        df4[col] = df4[col].astype(pd.Int64Dtype())

df4.dtypes

Year                   object
quarter                 int64
citymarketid_1         object
citymarketid_2         object
city1                  object
city2                  object
cur_passengers          Int64
cur_fare              float64
ly_fare               float64
ly_passengers           Int64
amount_change         float64
percent_change        float64
amount_change_pax       Int64
percent_change_pax    float64
dtype: object

In [13]:
df5['carpax'] = df5['carpax'].astype(str).str.replace(',', '', regex=False)
df5['carpax'] = pd.to_numeric(df5['carpax'], errors='coerce')
df5['carpax'] = df5['carpax'].astype(int)
df5['carpax'].dtypes

dtype('int64')

In [14]:
df7['TotalFaredPax'] = df7['TotalFaredPax'].astype(str).str.replace(',', '', regex=False).astype(int)

df7_num_cols = ['SHPax', 'SHAvgHubFare', 'SHPerPrem', 'LHPax', 'LHAvgHubFare', 'LHPerPrem']
# remove ',' with '' & 'NaN', 'NA', 'inf' with np.nan
for col in df7_num_cols:
    if any(x in col for x in ['SHPax', 'LHPax']):
        df7[col] = df7[col].str.replace(',', '', regex=False)
        df7[col] = pd.to_numeric(df7[col], errors='coerce').astype(pd.Int64Dtype())
    else:
        df7[col] = df7[col].astype(str).str.replace(',', '', regex=False)
        df7[col] = df7[col].replace(['NaN', 'NA', np.inf, -np.inf], np.nan)
        df7[col] = pd.to_numeric(df7[col], errors='coerce')
            
df7.dtypes

Year                 int64
quarter              int64
citymarketid         int64
TotalFaredPax        int64
TotalPerLFMkts     float64
TotalAvgHubFare    float64
TotalPerPrem       float64
SHPax                Int64
SHAvgHubFare       float64
SHPerPrem          float64
LHPax                Int64
LHAvgHubFare       float64
LHPerPrem          float64
dtype: object

## handling missing values

### checking if the data is normally distributed

In [15]:
# df1_num_cols = df1[['nsmiles', 'passengers', 'fare', 'large_ms', 'fare_lg', 'lf_ms', 'fare_low']]
# check_normality(df1_num_cols)

In [16]:
# df1a_num_cols = df1a[['large_ms', 'fare_lg', 'lf_ms', 'fare_low']]
# check_normality(df1a_num_cols)

In [17]:
# df7_num_cols = df7[['SHPax', 'SHAvgHubFare', 'SHPerPrem', 'LHPax', 'LHAvgHubFare', 'LHPerPrem']]
# check_normality(df7_num_cols)

### drop missing values for df1 and df2

In [18]:
df1.dropna(subset=['carrier_low', 'lf_ms', 'fare_low'], inplace=True)

In [19]:
df2.dropna(subset=['ly_passengers', 'ly_fare', 'ly_yield', 'ly_distance'], inplace=True)

In [20]:
df1a.isna().sum()

Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
nsmiles              0
passengers           0
fare                 0
carrier_lg        1550
large_ms          1550
fare_lg           1550
carrier_low       1622
lf_ms             1622
fare_low          1622
dtype: int64

### impute Iterative Imputer for missing values in normal distributions

In [21]:
df1a.isna().sum()

Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
nsmiles              0
passengers           0
fare                 0
carrier_lg        1550
large_ms          1550
fare_lg           1550
carrier_low       1622
lf_ms             1622
fare_low          1622
dtype: int64

In [22]:
df1a_imputed_cols = ['lf_ms', 'large_ms', 'fare_lg', 'fare_low']
num_imputer = IterativeImputer(random_state=0, max_iter=20, tol=1e-3)

scaler = StandardScaler()
df1a[df1a_imputed_cols] = scaler.fit_transform(df1a[df1a_imputed_cols])

df1a_imputed_values = num_imputer.fit_transform(df1a[df1a_imputed_cols])

df1a_imputed_df = pd.DataFrame(df1a_imputed_values, columns=df1a_imputed_cols, index=df1a.index)
df1a[df1a_imputed_cols] = scaler.inverse_transform(df1a_imputed_df)

print(df1a.dtypes)
df1a.isna().sum()

Year                int64
quarter             int64
citymarketid_1      int64
citymarketid_2      int64
city1              object
city2              object
nsmiles           float64
passengers          int64
fare              float64
carrier_lg         object
large_ms          float64
fare_lg           float64
carrier_low        object
lf_ms             float64
fare_low          float64
dtype: object


Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
nsmiles              0
passengers           0
fare                 0
carrier_lg        1550
large_ms             0
fare_lg              0
carrier_low       1622
lf_ms                0
fare_low             0
dtype: int64

In [23]:
df7.isna().sum()

Year                 0
quarter              0
citymarketid         0
TotalFaredPax        0
TotalPerLFMkts       0
TotalAvgHubFare      0
TotalPerPrem         0
SHPax              303
SHAvgHubFare       303
SHPerPrem          303
LHPax              298
LHAvgHubFare       298
LHPerPrem          298
dtype: int64

In [24]:
# impute missing values
df7_imputed_cols = ['SHPax', 'SHAvgHubFare', 'SHPerPrem', 'LHPax', 'LHAvgHubFare', 'LHPerPrem']
num_imputer = IterativeImputer(random_state=0, max_iter=20, tol=1e-3)

scaler = StandardScaler()
df7[df7_imputed_cols] = scaler.fit_transform(df7[df7_imputed_cols])

df7_imputed_values =num_imputer.fit_transform(df7[df7_imputed_cols])

df7_imputed_df = pd.DataFrame(df7_imputed_values, columns=df7_imputed_cols, index=df7.index)
df7[df7_imputed_cols] = scaler.inverse_transform(df7_imputed_df)

# convert passenger columns to int 
df7['SHPax'] = df7['SHPax'].astype(int)
df7['LHPax'] = df7['LHPax'].astype(int)

print(df7.dtypes)
df7.isna().sum()

Year                 int64
quarter              int64
citymarketid         int64
TotalFaredPax        int64
TotalPerLFMkts     float64
TotalAvgHubFare    float64
TotalPerPrem       float64
SHPax                int64
SHAvgHubFare       float64
SHPerPrem          float64
LHPax                int64
LHAvgHubFare       float64
LHPerPrem          float64
dtype: object


Year               0
quarter            0
citymarketid       0
TotalFaredPax      0
TotalPerLFMkts     0
TotalAvgHubFare    0
TotalPerPrem       0
SHPax              0
SHAvgHubFare       0
SHPerPrem          0
LHPax              0
LHAvgHubFare       0
LHPerPrem          0
dtype: int64

### categorical data

#### impute with the most frequent category

In [25]:
df1a['carrier_lg'] = df1a['carrier_lg'].replace('nan', np.nan)
df1a['carrier_low'] = df1a['carrier_low'].replace('nan', np.nan)

cat_imputer = SimpleImputer(strategy="most_frequent")
df1a['carrier_lg'] = cat_imputer.fit_transform(df1a[['carrier_lg']]).ravel()
df1a['carrier_low'] = cat_imputer.fit_transform(df1a[['carrier_low']]).ravel()
df1a.isna().sum()

Year              0
quarter           0
citymarketid_1    0
citymarketid_2    0
city1             0
city2             0
nsmiles           0
passengers        0
fare              0
carrier_lg        0
large_ms          0
fare_lg           0
carrier_low       0
lf_ms             0
fare_low          0
dtype: int64

In [26]:
df1.to_csv('../datasets/processed/df1.csv', index=False)
df1a.to_csv('../datasets/processed/df1a.csv', index=False)
df2.to_csv('../datasets/processed/df2.csv', index=False)
df3.to_csv('../datasets/processed/df3.csv', index=False)
df4.to_csv('../datasets/processed/df4.csv', index=False)
df5.to_csv('../datasets/processed/df5.csv', index=False)
df7.to_csv('../datasets/processed/df7.csv', index=False)

print('dfs has been saved successfully!')

dfs has been saved successfully!


In [27]:
def clean_numeric_col(series):
    if series.dtype == 'object':
        series = series.str.replace('%', '', regex=False)
        series = series.str.replace(',', '.', regex=False) 
        series = series.str.replace(',', '', regex=False)
    numeric_series = pd.to_numeric(series, errors='coerce')
    return numeric_series

In [28]:
df_t1 = pd.read_csv('../datasets/processed/df1.csv', sep=',', decimal='.')
df_t7 = pd.read_csv('../datasets/processed/df7.csv', sep=',', decimal='.')

In [29]:
print("--- Inspecting df_t1['citymarketid_1'] before cleaning/conversion ---")
print("Data Type:", df_t1['citymarketid_1'].dtype)
print("Non-Null Count:", df_t1['citymarketid_1'].notna().sum())
print("Unique values (sample):")
try:
    unique_vals = df_t1['citymarketid_1'].unique()
    print(unique_vals[:50]) # shows values like '32,575'
except Exception as e:
    print(f"Could not display unique values: {e}")


# --- Clean the string column BEFORE converting ---
print("\n--- Applying cleaning steps to citymarketid_1 ---")

# 1. Ensure it's treated as string and strip whitespace 
df_t1['citymarketid_1'] = df_t1['citymarketid_1'].astype(str).str.strip()

# 2. Remove commas used as thousands separators
print("Removing commas...")
df_t1['citymarketid_1'] = df_t1['citymarketid_1'].str.replace(',', '', regex=False)

# Check unique values again AFTER cleaning to verify
print("Unique values (sample) AFTER cleaning:")
unique_vals_cleaned = df_t1['citymarketid_1'].unique()
print(unique_vals_cleaned[:50])

# --- Convert to numeric AFTER cleaning ---
print("\n--- Attempting Conversion to Int64 ---")
df_t1['citymarketid_1'] = pd.to_numeric(df_t1['citymarketid_1'], errors='coerce').astype('Int64')
# Also convert citymarketid_2 and df_t7['citymarketid']
df_t1['citymarketid_2'] = pd.to_numeric(df_t1['citymarketid_2'], errors='coerce').astype('Int64')
df_t7['citymarketid'] = pd.to_numeric(df_t7['citymarketid'], errors='coerce').astype('Int64')


# --- Check conversion result ---
print("\n--- Checking df_t1['citymarketid_1'] AFTER conversion ---")
print("Data Type:", df_t1['citymarketid_1'].dtype)
print("Non-Null Count:", df_t1['citymarketid_1'].notna().sum())
print("Null Count:", df_t1['citymarketid_1'].isna().sum()) 


# --- Verify types before merge ---
print("\n--- Verifying types before merge ---")
print(f"df_t1 citymarketid_1 dtype: {df_t1['citymarketid_1'].dtype}, Non-Nulls: {df_t1['citymarketid_1'].notna().sum()}")
print(f"df_t1 citymarketid_2 dtype: {df_t1['citymarketid_2'].dtype}, Non-Nulls: {df_t1['citymarketid_2'].notna().sum()}")
print(f"df_t7 citymarketid dtype: {df_t7['citymarketid'].dtype}, Non-Nulls: {df_t7['citymarketid'].notna().sum()}")


# --- Proceed with merge only if conversion was successful ---
if df_t1['citymarketid_1'].notna().sum() > 0:
    print("\nConversion appears successful, proceeding with merge...")

    # Ensure 'Year', 'quarter' have matching types
    df_t1['Year'] = pd.to_numeric(df_t1['Year'], errors='coerce').astype('Int64')
    df_t1['quarter'] = pd.to_numeric(df_t1['quarter'], errors='coerce').astype('Int64')
    df_t7['Year'] = pd.to_numeric(df_t7['Year'], errors='coerce').astype('Int64')
    df_t7['quarter'] = pd.to_numeric(df_t7['quarter'], errors='coerce').astype('Int64')

    # Clean T7 columns and rename (using correct T7 column names from info())
    df_t7_clean = df_t7[['Year', 'quarter', 'citymarketid', 'TotalPerPrem', 'TotalPerLFMkts']].copy()
    df_t7_clean.rename(columns={'TotalPerPrem': 'city_fare_premium',
                          'TotalPerLFMkts': 'city_perc_lcc_pax'}, inplace=True)
    df_t7_final = df_t7_clean[['Year', 'quarter', 'citymarketid', 'city_fare_premium', 'city_perc_lcc_pax']]

print(f"Number of rows in df_t7_final before duplicate check: {len(df_t7_final)}")
key_cols = ['Year', 'quarter', 'citymarketid']

# --- Check for duplicates based on merge keys ---
num_duplicates = df_t7_final.duplicated(subset=key_cols).sum()
print(f"Number of duplicate key sets found in df_t7_final: {num_duplicates}")

if num_duplicates > 0:
    print("Handling duplicates by averaging values...")
    value_cols = ['city_fare_premium', 'city_perc_lcc_pax'] 
    
    # Group by keys, calculate mean, reset index
    df_t7_final = df_t7_final.groupby(key_cols, as_index=False)[value_cols].mean()
    print(f"Number of rows in df_t7_final after averaging duplicates: {len(df_t7_final)}")

    # Merge for Origin City
    df_merged = pd.merge(df_t1, df_t7_final,
                         left_on=['Year', 'quarter', 'citymarketid_1'],
                         right_on=['Year', 'quarter', 'citymarketid'],
                         how='left')
    df_merged.rename(columns={'city_fare_premium': 'origin_fare_premium',
                              'city_perc_lcc_pax': 'origin_perc_lcc_pax'}, inplace=True)

    # Merge for Destination City
    df_merged = pd.merge(df_merged, df_t7_final,
                         left_on=['Year', 'quarter', 'citymarketid_2'],
                         right_on=['Year', 'quarter', 'citymarketid'],
                         how='left', suffixes=('_orig_temp', ''))
    df_merged.rename(columns={'city_fare_premium': 'dest_fare_premium',
                              'city_perc_lcc_pax': 'dest_perc_lcc_pax'}, inplace=True)

    # Clean up columns
    df_merged.drop(columns=['citymarketid_orig_temp', 'citymarketid'], errors='ignore', inplace=True)

    # Create route_id
    df_merged['route_id'] = df_merged['citymarketid_1'].astype(str) + '_' + df_merged['citymarketid_2'].astype(str)

    # Assign to df and check results
    df = df_merged.copy()
    print("\n--- Final merged DataFrame info: ---")
    df.info()
    print("\n--- Final missing values AFTER MERGE: ---")
    print(df.isnull().sum())

    # Handle remaining NAs (from non-matches)
    fill_cols = ['origin_fare_premium', 'origin_perc_lcc_pax', 'dest_fare_premium', 'dest_perc_lcc_pax']
    for col in fill_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0) # fill with 0 for no matching city market ids
    
    print("\n--- Final missing values AFTER HANDLING: ---")
    print(df.isnull().sum())

else:
    print("\nMerge skipped: df_t1['citymarketid_1'] conversion still failed after cleaning.")
    print("If commas were removed and it still fails, re-inspect unique values for other issues.")

--- Inspecting df_t1['citymarketid_1'] before cleaning/conversion ---
Data Type: object
Non-Null Count: 115032
Unique values (sample):
['32,575' '32,467' '31,703' '30,977' '30,397' '32,211' '30,721' '32,457'
 '30,194' '33,570' '30,325' '31,454' '31,453' '31,057' '31,295' '30,693'
 '30,466' '34,057' '31,650' '30,559' '30,423' '33,195' '33,192' '31,714'
 '30,994' '33,495' '31,136' '30,792' '34,614' '32,337' '31,066' '30,647'
 '34,100' '31,135' '33,105' '30,529' '31,123' '33,198' '30,713' '33,342'
 '33,214' '30,852' '34,492' '30,140' '34,321' '30,154' '30,198' '33,044'
 '31,995' '33,244']

--- Applying cleaning steps to citymarketid_1 ---
Removing commas...
Unique values (sample) AFTER cleaning:
['32575' '32467' '31703' '30977' '30397' '32211' '30721' '32457' '30194'
 '33570' '30325' '31454' '31453' '31057' '31295' '30693' '30466' '34057'
 '31650' '30559' '30423' '33195' '33192' '31714' '30994' '33495' '31136'
 '30792' '34614' '32337' '31066' '30647' '34100' '31135' '33105' '30529'
 '3112

In [30]:
print("--- Adding Holiday Features ---")

df['Year_n'] = df['Year'].astype(int)

# Define major holidays/periods often affecting travel
# (Thanksgiving and Christmas fall in Q4, New Year's Day in Q1, Memorial Day in Q2, July 4th/Labor Day in Q3)

# Get US holidays for the range of years in the data
min_year = df['Year_n'].min()
max_year = df['Year_n'].max()
try:
    us_holidays = holidays.US(years=range(min_year, max_year + 1))
    print(f"Generated US holidays from {min_year} to {max_year}")

    # Function to check if key holidays fall within a quarter
    def check_major_holidays(row):
        year = row['Year_n']
        quarter = row['quarter']
        
        # Q1: New Year's Day
        has_nyd = 1 if quarter == 1 and pd.Timestamp(f'{year}-01-01') in us_holidays else 0
        
        # Q2: Memorial Day (Last Monday in May) - Check if May is in Q2
        has_memorial = 1 if quarter == 2 and any(date.month == 5 and holiday == 'Memorial Day' for date, holiday in us_holidays.items() if date.year == year) else 0
        
        # Q3: July 4th, Independence Day - Check if Jul/Aug/Sep are in Q3
        has_jul4 = 1 if quarter == 3 and \
            (pd.Timestamp(f'{year}-07-04') in us_holidays or \
             any(date.month == 9 and holiday == 'Independence Day' for date, holiday in us_holidays.items() if date.year == year)
            ) else 0
        
        # Q3: Labor Day (First Monday in Sep) - Check if Jul/Aug/Sep are in Q3
        has_labor = 1 if quarter == 3 and \
            (pd.Timestamp(f'{year}-09-01') in us_holidays or \
             any(date.month == 9 and holiday == 'Labor Day' for date, holiday in us_holidays.items() if date.year == year) 
            ) else 0
        
        # Q4: Thanksgiving (Fourth Thursday in Nov), Christmas Day
        nov_dates = pd.date_range(start=f'{year}-11-01', end=f'{year}-11-30', freq='D')
        thus = nov_dates[nov_dates.weekday == 3]
        thx_date = thus[3] if len(thus) >= 4 else None
        has_thx = 1 if quarter == 4 and (thx_date is not None and thx_date in us_holidays) else 0
        
        # Q4: Christmas Day (Dec 25)
        has_christmas = 1 if quarter == 4 and pd.Timestamp(f'{year}-12-25') in us_holidays else 0

        # Flag if *any* federal holiday falls in the quarter
        has_any = 0
        try:
            # Define quarter start and end dates accurately
            q_start_month = (quarter - 1) * 3 + 1
            q_end_month = quarter * 3
            start_date = pd.Timestamp(f'{year}-{q_start_month:02d}-01')
            if q_end_month == 12:
                end_date = pd.Timestamp(f'{year}-12-31')
            else:
                next_q_start_month = q_end_month + 1
                end_date = pd.Timestamp(f'{year}-{next_q_start_month:02d}-01') - pd.Timedelta(days=1)

            # Check if any holiday falls within the quarter range
            for holiday_date in us_holidays:
                if holiday_date.year == year:
                    holiday_ts = pd.Timestamp(holiday_date)
                    if start_date <= holiday_ts <= end_date:
                        has_any = 1
                        break # found one, no need to check further for this quarter
        except Exception as e:
            has_any = 0 # default to 0 if range check fails

        return pd.Series([has_nyd, has_memorial, has_jul4, has_labor, has_thx, has_christmas, has_any])


except Exception as e:
    print(f"Error initializing holidays or defining function: {e}")
    def check_major_holidays(row):
        return pd.Series([0, 0, 0, 0, 0, 0, 0])
    
# Apply the function row-wise
print("Applying holiday checks...")
holiday_cols = ['is_Q_with_NYD', 'is_Q_with_Memorial', 'is_Q_with_Jul4_Indep', 'is_Q_with_Labor', 'is_Q_with_Thx', 'is_Q_with_Christmas', 'is_Q_with_Any_Holiday']
df[holiday_cols] = df.apply(check_major_holidays, axis=1, result_type='expand')

# Drop the temporary numeric year column
df.drop(columns=['Year_n'], inplace=True)

print("\nHoliday features added:")
print(df[holiday_cols].sum()) # show how many rows have these flags set
df.head()

--- Adding Holiday Features ---
Generated US holidays from 1996 to 2024
Applying holiday checks...

Holiday features added:
is_Q_with_NYD             29006
is_Q_with_Memorial        29008
is_Q_with_Jul4_Indep      29007
is_Q_with_Labor           29007
is_Q_with_Thx             28011
is_Q_with_Christmas       28011
is_Q_with_Any_Holiday    115032
dtype: int64


,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,...,dest_fare_premium,dest_perc_lcc_pax,route_id,is_Q_with_NYD,is_Q_with_Memorial,is_Q_with_Jul4_Indep,is_Q_with_Labor,is_Q_with_Thx,is_Q_with_Christmas,is_Q_with_Any_Holiday
0,2024,3,32575,32457,"Los Angeles, CA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",372,16339,159.59,WN,...,-0.031567,0.891767,32575_32457,0,0,1,1,0,0,1
1,2024,3,32467,31703,"Miami, FL (Metropolitan Area)","New York City, NY (Metropolitan Area)",1118,15842,179.57,B6,...,0.033580,0.811460,32467_31703,0,0,1,1,0,0,1
2,2024,3,32575,31703,"Los Angeles, CA (Metropolitan Area)","New York City, NY (Metropolitan Area)",2510,13767,399.68,B6,...,0.033580,0.811460,32575_31703,0,0,1,1,0,0,1
3,2024,3,31703,31454,"New York City, NY (Metropolitan Area)","Orlando, FL",989,12511,176.63,B6,...,-0.140200,0.969300,31703_31454,0,0,1,1,0,0,1
4,2024,3,30977,31703,"Chicago, IL","New York City, NY (Metropolitan Area)",773,11466,184.79,UA,...,0.033580,0.811460,30977_31703,0,0,1,1,0,0,1


In [31]:
df.to_csv('../datasets/processed/base_df.csv', index=False)